Desempleo, total (Como porcentaje de la fuerza Laboral<br>
extraido WDB - "SL.UEM.TOTL.ZS" - DB=2

In [1]:
#importo librerias
import wbgapi as wb
import pandas as pd

In [2]:
#Filtro por pais del Continente Americano
paises = ['ATG','ARG','BRB','BLZ','BOL','BRA','CAN','CHL','COL','CRI','CUB','DMA','DOM','ECU','SLV','GRD','GTM','GUY','HTI','HND','JAM','MEX','NIC','PAN','PRY','PER','KNA','LCA','VCT','SUR','BHS','TTO','USA','URY','VEN']

In [3]:
#POBLACION ACTIVA (FUERZA LABORAL) - extraido de WDB - "SL.TLF.TOTL.IN" - DB=2
#CREACION DE DATAFRAME
wb.db= 2
tbl_desempleo_fl = pd.read_csv('Extraccion/tbl_desempleo_fl.csv')

In [4]:
tbl_desempleo_fl
#Pais_ID	Year	Desempleo

,id_pais,year,desempleo
0,VEN,2020,6.627
1,VEN,2019,5.096
2,VEN,2018,5.022
3,VEN,2017,5.050
4,VEN,2016,5.320
...,...,...,...
730,ATG,2004,NaN
731,ATG,2003,NaN
732,ATG,2002,NaN
733,ATG,2001,NaN


In [5]:
tbl_desempleo_fl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_pais    735 non-null    object 
 1   year       735 non-null    int64  
 2   desempleo  651 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 17.4+ KB


In [6]:
#CONTAMOS VALORES NULOS
tbl_desempleo_fl.isnull().sum()

id_pais       0
year          0
desempleo    84
dtype: int64

HaY 84 datos faltantes, algunos paises tienen todos sus datos faltantes<br>
Vamos a identificarlos

In [7]:
#Vemos que paises tienen TODOS sus datos faltantes
print("Paises con TODOS sus datos nulos")
paises_nulos=[]
for i in paises:
    mask = tbl_desempleo_fl["id_pais"] == i
    df_filtrada = tbl_desempleo_fl[mask]
    a = df_filtrada.iloc[:,2].isnull().sum()
    if a == 21:
        paises_nulos.append(i)
print(paises_nulos)

Paises con TODOS sus datos nulos
['ATG', 'DMA', 'GRD', 'KNA']


LOS 84 DATOS FALTANTES CORRESPONDEN A CUATROS PAISES:<br>
ATG, DMA, GRD, KNA<br>
VAMOS A BUSCAR LA TASA DE DESEMPLEO para el 2020 y,<br>
VAMOS A LLENAR LOS DATOS FALTANTES POR MEDIO DE COMPARACION CON PAISES SIMILARES<br>

Las tasa de desempleo año 2020 de los paises con datos nulos son:<br>
ATG= 11%<br>
DMA= 23%<br>
GRD= 24%<br>
KNA= 4.5%<br>

Fuente: https://www.datosmundial.com/desempleo.php

En cuanto a la variacion de la tasa de desempleo, utilizaremos LCA como referencia

In [8]:
tbl_desempleo_fl[tbl_desempleo_fl["id_pais"] == "LCA"]

,id_pais,year,desempleo
147,LCA,2020,16.892000
148,LCA,2019,15.320000
149,LCA,2018,19.080000
150,LCA,2017,18.860001
151,LCA,2016,19.649000
152,LCA,2015,20.562000
153,LCA,2014,21.296000
154,LCA,2013,22.209999
155,LCA,2012,19.059999
156,LCA,2011,18.860001


In [10]:
tbl_desempleo_fl.columns

Index(['id_pais', 'year', 'desempleo'], dtype='object')

In [11]:
#creamos un dataframe con los valores a cargar en el año 2020
imp=[11.00, 23.00, 24.00, 4.50]
df_carga=pd.DataFrame(list(zip(paises_nulos, imp)), columns= ["id_pais","desempleo"])
df_carga


,id_pais,desempleo
0,ATG,11.0
1,DMA,23.0
2,GRD,24.0
3,KNA,4.5


In [12]:
#Cargamos los valores en cada celda correspondiente al pais y año
for i in paises_nulos:
    mask1 = (tbl_desempleo_fl["id_pais"] == i) & (tbl_desempleo_fl["year"] == 2020)
    mask2 = df_carga["id_pais"] == i
    tbl_desempleo_fl.loc[mask1, "desempleo"] = df_carga["desempleo"].loc[mask2].values[0]
    print (tbl_desempleo_fl[mask1])

    id_pais  year  desempleo
714     ATG  2020       11.0
    id_pais  year  desempleo
483     DMA  2020       23.0
    id_pais  year  desempleo
399     GRD  2020       24.0
    id_pais  year  desempleo
168     KNA  2020        4.5


In [13]:
#DEBERIA HABER 4 DATOS NULOS MENOS (80)
tbl_desempleo_fl.isnull().sum()

id_pais       0
year          0
desempleo    80
dtype: int64

CALCULAREMOS EL INCREMENTO O DECREMENTO ANUAL PARA "LCA" y LO APLICAREMOS A LOS OTROS PAISES

In [14]:
#Creamos un dataframe con la tasa de desempleo de LCA
df1=tbl_desempleo_fl[tbl_desempleo_fl["id_pais"] == "LCA"].reset_index()
df1.drop(["index","id_pais","year"], axis=1, inplace=True)
df1

,desempleo
0,16.892000
1,15.320000
2,19.080000
3,18.860001
4,19.649000
5,20.562000
6,21.296000
7,22.209999
8,19.059999
9,18.860001


In [15]:
#Creamos una lista que indique la variacion inter anual comenzando con el año 2020 para LCA para abajo
var_anual=[]
for i in range (1, len(df1)):
    val_ant=df1.loc[i-1,"desempleo"]
    val_act=df1.loc[i,"desempleo"]
    var_anual.append(round((val_act/val_ant),2))
print(var_anual)

[0.91, 1.25, 0.99, 1.04, 1.05, 1.04, 1.04, 0.86, 0.99, 0.91, 0.9, 0.83, 1.1, 1.14, 1.17, 1.12, 1.06, 0.92, 0.91, 0.89]


In [17]:
for i in paises_nulos:
    year_act=2019
    year_prev=2020
    for e in var_anual:
        mask1= (tbl_desempleo_fl["id_pais"] == i) & (tbl_desempleo_fl["year"] == year_act)
        mask2= (tbl_desempleo_fl["id_pais"] == i) & (tbl_desempleo_fl["year"] == year_prev)
        val=tbl_desempleo_fl["desempleo"].loc[mask2].values[0] * e
        tbl_desempleo_fl.loc[mask1, "desempleo"] = round(val, 3)
        year_act= int(year_act)
        year_act = year_act-1
        year_prev= int(year_prev)
        year_prev = year_prev-1

In [18]:
tbl_desempleo_fl[tbl_desempleo_fl["id_pais"] == "ATG"]

,id_pais,year,desempleo
714,ATG,2020,11.000
715,ATG,2019,10.010
716,ATG,2018,12.512
717,ATG,2017,12.387
718,ATG,2016,12.882
719,ATG,2015,13.526
720,ATG,2014,14.067
721,ATG,2013,14.630
722,ATG,2012,12.582
723,ATG,2011,12.456


In [19]:
tbl_desempleo_fl[tbl_desempleo_fl["id_pais"] == "DMA"]

,id_pais,year,desempleo
483,DMA,2020,23.000
484,DMA,2019,20.930
485,DMA,2018,26.162
486,DMA,2017,25.900
487,DMA,2016,26.936
488,DMA,2015,28.283
489,DMA,2014,29.414
490,DMA,2013,30.591
491,DMA,2012,26.308
492,DMA,2011,26.045


In [20]:
#CORROBORAMOS DATOS NULOS
tbl_desempleo_fl.isnull().sum()

id_pais      0
year         0
desempleo    0
dtype: int64

In [21]:
tbl_desempleo_fl

,id_pais,year,desempleo
0,VEN,2020,6.627
1,VEN,2019,5.096
2,VEN,2018,5.022
3,VEN,2017,5.050
4,VEN,2016,5.320
...,...,...,...
730,ATG,2004,13.915
731,ATG,2003,14.750
732,ATG,2002,13.570
733,ATG,2001,12.349
